# Optimizadores en Pytorch

En este  notebook vamos a prestar una especial atención a los optimizadores en Pytorch. 

Los optimizadores se encuentran agrupados dentro del paquete torch.optim. Su rol dentro del proceso de aprendizaje es integrarse con el mecanismo de autograd de los tensores y proporcionar una implementación de la pasada *backward*. Para ello, tienen que obtener información de cómo de lejos está el modelo de la salida deseada para realizar un cálculo de los gradientes a aplicar sobre los parámetros del modelo. Las funciones de pérdida se tratan de forma separada en otro notebook.

Algunas características comunes a los optimizadores de Pytorch son:

- Mantienen el estado actual y actualizan los parámetros usando un cálculo de sus gradientes
- Tienen una interfaz común que facilita 
    - Que sean fácilmente intercambiables
    - Que se pueden implementar optimizadores ad-hoc
- Reciben un iterables con los parámetros aprendibles del modelo
    - También reciben otros parámetros, llamados hiperparámetros del optimizador, que permite su configuración (ejs. learning rate, momentum)

Métodos principales:

- *backward()*: Calcula los gradientes. Se aplica a la función de pérdida, no al optimizador.
- *zero_grad()*: Pone a 0 los gradientes del optimizador
- *step()*: Aplica los gradientes calculados por *backward()* a los parámetros del modelo

La clase base del optimizador es optim.Optimizer y recibe como parámetros:
- Un interable que contiene los parámetros a optimizar
- Un diccionario con los hiperparámetros del optimizador (lr, momentum, etc...)

A continuación iremos viendo ejemplos ilustrativos del uso de los optimizadores más comunes: SGD, RMSProp, Adagrad, Adam, Adadelta.

Empecemos por Stochastic Gradiente Descent (SGD)

In [7]:
# Importamos las bibliotecas necesarias
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F

# Definimos las transformaciones para los conjuntos de datos
transform = transforms.Compose([
    # Transformamos las imágenes a tensores
    transforms.ToTensor(),
    # Normalizamos los tensores con media 0.5 y desviación estándar 0.5
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# Cargamos los conjuntos de datos CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Definimos una red convolucional simple
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 3 canales de entrada para RGB, 6 de salida, kernel de 5x5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 clases de salida para CIFAR-10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

# Especificamos la función de pérdida y el optimizador
# En CrossEntropyLoss, el mejor valor de la función de pérdida es 0
criterion = nn.CrossEntropyLoss()
# SGD: Stochastic Gradient Descent
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, foreach=False, nesterov=False)

# Finalmente, entrenamos el modelo
# Durante 10 épocas
for epoch in range(10):
    running_loss = 0.0
    # Para cada lote de datos
    for i, data in enumerate(trainloader, 0):
        # Obtenemos las entradas y las etiquetas del lote del conjunto de entrenamiento
        inputs, labels = data
        # Reiniciamos los gradientes
        optimizer.zero_grad()
        # Hacemos una pasada hacia adelante
        outputs = model(inputs)
        # Calculamos la pérdida
        loss = criterion(outputs, labels)
        # Hacemos una pasada hacia atrás
        loss.backward()
        # Actualizamos los parámetros
        optimizer.step()
        # Imprimimos estadísticas
        running_loss += loss.item()
        
    print(f'Época {epoch+1}, Pérdida: {running_loss/len(trainloader)}')


Files already downloaded and verified
Files already downloaded and verified
Época 1, Pérdida: 1.7768517238709627
Época 2, Pérdida: 1.3932885832493873
Época 3, Pérdida: 1.2550515693319424
Época 4, Pérdida: 1.158853273943562
Época 5, Pérdida: 1.0884042653586248
Época 6, Pérdida: 1.0276772107004815
Época 7, Pérdida: 0.973789001745946
Época 8, Pérdida: 0.9205946205445873
Época 9, Pérdida: 0.8784698215134613
Época 10, Pérdida: 0.8438497624738747


En la celda anterior, podemos hacer distintas ejecuciones cambiando los valores de los parámetros de la celda de construcción del optimizador:

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, foreach=False, nesterov=False)
